In [25]:
import requests
import boto3
from mypy_boto3_s3 import S3Client
from pprint import pprint
import numpy as np
import json

In [46]:
API_BASE_URL="http://localhost:8000"
BUCKET="cow-detect-maia"

app_info = requests.get(f"{API_BASE_URL}/app-info").json()
pprint(app_info)
model_name = app_info["model_info"]["model_metadata"]["name"]
model_arch = app_info["model_info"]["model_metadata"].get("backbone_arch", None)
# region = {"HerdNet": "Chad-HN", "FasterRCNNResNetFPN": "Chad-FRC"}[model_name]
region = "Time-Measurement"

print(f"{model_name=}  region: {region}")


{'model_info': {'cfg_path': 'configs/test/herdnet.yaml',
                'model_metadata': {'cfg_path': 'configs/test/herdnet.yaml',
                                   'class_name': 'LossWrapper',
                                   'name': 'HerdNet',
                                   'num_classes': 7,
                                   'patch_size': 512,
                                   'weights_path': 'data/models/herdnet_v3/best_model.pth'},
                'weights_path': 'data/models/herdnet_v3/best_model.pth'},
 's3_bucket': 'cow-detect-maia'}
model_name='HerdNet'  region: Time-Measurement


In [10]:
session = boto3.Session(region_name="us-east-1", profile_name="dvc-user")
s3: S3Client = session.client("s3")


In [11]:
prefix = f"{region}/2025-11-22/"
s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix)['Contents'][0]['Key']

'Time-Measurement/2025-11-22/2af4192ee4de23185abb0e1677b61fd9ea52db61.JPG'

In [51]:
payload = {"s3_path": f"s3://{BUCKET}/Time-Measurement/2025-11-22/2af4192ee4de23185abb0e1677b61fd9ea52db61.JPG"}
times = []
for _ in range(8):
    resp = requests.post(f"{API_BASE_URL}/predict", json=payload)
    inf_time = resp.json()['inference_time']
    print(inf_time)
    times.append(inf_time)

print(times)

4.915628012968227
3.833197708008811
4.583362037956249
4.4171896949992515
3.9724179710028693
4.372124302957673
3.908080536988564
4.272355091001373
[4.915628012968227, 3.833197708008811, 4.583362037956249, 4.4171896949992515, 3.9724179710028693, 4.372124302957673, 3.908080536988564, 4.272355091001373]


In [35]:
# FRC-50 cuda : NVIDIA GeForce RTX 2060 (6.0 GB de VRAM)
times = [7.45, 7.11, 7.07, 7.30, 7.34, 7.46, 7.12, 7.28]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

memory = 2.8   # GB

7.27 +/- 0.14


In [34]:
# FRC-101 cuda:
times = [9.66, 9.20, 9.30, 9.137716260971501, 9.3829946849728, 9.237513903004583, 9.202632041997276, 9.416010013024788]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))
memory = 2.9  # GB

9.32 +/- 0.16


In [41]:
# HN-2-hn cuda:
times = [4.369541400985327, 3.7918004079838283, 3.7768040589871816, 4.021001018001698, 5.97066516103223, 4.028331107983831, 4.044117910030764, 3.852164224954322]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

memory = 2.6 # GB

4.23 +/- 0.68


In [48]:
# HN-2
times = [4.634246086003259, 3.932469096966088, 4.064917451993097, 4.624724559020251, 3.981831144017633, 4.66037645196775, 4.679108470038045, 4.5809408979839645]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

memory = 2.6 # GB

4.39 +/- 0.31


In [49]:
# HN-3
times = [4.883444601029623, 3.918730864999816, 3.9554799090255983, 4.147527578053996, 4.240999165980611, 4.01755037996918, 4.255680132017005, 4.2202754089958034]
print( np.round(np.mean(times), 2), '+/-', np.round(np.std(times), 2))

4.2 +/- 0.28
